In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("/home/bis/2021_SJH_detectability/Detectability")
from utils import *

In [2]:
df_kb = pd.read_csv('../data/massIVE-KB/df_kb_filter.csv')
df_uni = pd.read_csv('../data/uniprot/df_uni.csv')

In [3]:
df_kb.head(1)

,Unnamed: 0,PEPMASS,CHARGE,FILENAME,SEQ,PROTEIN,STRIP_PEPTIDE,SCORE,MZ,INTENSITY,SPECTRAL_CNT,PROTEIN_except_decoy,PROTEIN_except_else,PTM_SEQ
0,0,623.327626,3,filtered_library_mgf_files/9187af535f6e41ce958...,+42.011AAEAAGGKYRSTVSKSKD,sp|Q96A65|EXOC4_HUMAN,AAEAAGGKYRSTVSKSKD,4.605175,129.10317993164062;130.08726501464844;136.0766...,63295.375;7010.876953125;14278.6826171875;5026...,1,sp|Q96A65|EXOC4_HUMAN,sp|Q96A65|EXOC4_HUMAN,(Ace)AAEAAGGKYRSTVSKSKD


# Guide
  - STRIP_PEPTIDE의 spectral count 정리 기준
    - PTM_PEPTIDE(SEQ)도 protein에서 ID된것이긴 하니까 STRIP_PEPTIDE 기준으로 detect 된 것으로 판단해야 한다.
  - point 1
      - PTM_PEPTIDE와 noPTM_PEPTIDE를 STRIP_PEPTIDE로 만들 때, spectral count 합쳐야 할까?  
        - protien 기준에서 id peptide를 판단할 때, PTM없이 판단하므로 합치는게 맞다.
      - 결론 : 맞다.
  - point 2
      - PTM_PEPTIDE 의 m/z, intensity 와 noPTM_PEPTIDE 의 m/z, intensity가 다를텐데 prosit 기준값은?
          - m/z, intensity 는 noPTM_PEPTIDE것으로 사용한다.
          - 물론, noPTM_PEPTIDE와 겹치지 않으면 STRIP_PEPTIDE의 스펙트럼 사용
      - 결론 : PTM_PEPTIDE의 STRIP_PEPTIDE서열이 noPTM에 있는 경우 noPTM 스펙트럼 사용  
        없는 경우, best score의 스펙트럼 사용
  - point 3
      - CHARGE 마다 spectral count 다른데, 합쳐야할까?
      - 1과 같은 의미로 합치는게 맞다.
      - 다만, m/z, intensity 비교를 위해 charge별 하나를 정해야하므로 별도로 유지한다.

In [4]:
# protein check
pc2prot = dict()
for pep, c, prot in df_kb[['STRIP_PEPTIDE', 'CHARGE', 'PROTEIN']].values:
    if (pep, c) not in pc2prot:
        pc2prot[(pep, c)]=set()
    for p in prot.split(';'):
        pc2prot[(pep, c)].add(p)
    
print('STRIP precursor which has protein >2 cnt : ', 
      len([_ for _ in pc2prot.values() if len(_) >=2]))

for (p, c), v in pc2prot.items():
    pc2prot[(p, c)] = ';'.join(list(v))

STRIP precursor which has protein >2 cnt :  71419


In [5]:
# ptm spectral count merge (plus)
# Nterm (Ace), (Cab), (Aml) 은 spectral count 1로 둬서 나중에 빠질 수 있게 구성 !! 
ptm_arr = ['(Ace)', '(Cab)', '(Aml)']
pc2cnt = dict()
for c, p, cnt in df_kb[['CHARGE', 'STRIP_PEPTIDE', 'SPECTRAL_CNT']].values:
    if (p, c) not in pc2cnt:
        pc2cnt[(p, c)] = 0
    pc2cnt[(p, c)] += cnt
    
for c, p, ptm_seq in df_kb[['CHARGE', 'STRIP_PEPTIDE', 'PTM_SEQ']].values:
    if sum([1 for _ in ptm_arr if _ in ptm_seq])>=1:
        pc2cnt[(p, c)]=1

In [6]:
# spectrum check
# 84만개의 no PTM PEPTIDE
pc2mz = dict()
pc2int = dict()
for p, c, m, i in df_kb[['SEQ', 'CHARGE', 'MZ', 'INTENSITY']].values:  
    if '+' not in p:
        pc2mz[(p, c)] = m
        pc2int[(p, c)] = i

# spectrum check
# 20만개의 PTM PEPTIDE
noptm_pc = {(_, c) for c, _ in df_kb[['CHARGE', 'SEQ']].values if '+' not in _}
whole_pc = {(_, c) for c, _ in df_kb[['CHARGE', 'STRIP_PEPTIDE']].values}
ptm_pc_candi = {k:set() for k in list(whole_pc.difference(noptm_pc))}

# PTM_PEPTIDE candidate
# ptm_pc_candi = (STRIP_PEPTIDE, CHARGE):(PTM_PEPTIDE, CHARGE)
for c, p, seq in df_kb[['CHARGE', 'STRIP_PEPTIDE', 'SEQ']].values:
    if (p, c) in ptm_pc_candi:
        ptm_pc_candi[(p, c)].add((seq, c))

pc2score_look = {(p, c):score for p, c, score in df_kb[['SEQ', 'CHARGE', 'SCORE']].values}
pc2mz_look = {(p, c):mz for p, c, mz in df_kb[['SEQ' ,'CHARGE', 'MZ']].values}
pc2int_look = {(p, c):i for p, c, i in df_kb[['SEQ' ,'CHARGE', 'INTENSITY']].values}

for k, v in ptm_pc_candi.items():
    if len(v)==1:
        v = list(v)[0]
        pc2mz[k] = pc2mz_look[v]
        pc2int[k] = pc2int_look[v]
    else:  # choose high SCORE's SEQ
        ptm_seqs = {ptm_seq:pc2score_look[ptm_seq] for ptm_seq in v}
        best_v = sorted(ptm_seqs.items(), key=lambda x: x[1], reverse=True)[0][0]  # 내림차순
        pc2mz[k] = pc2mz_look[best_v]
        pc2int[k] = pc2int_look[best_v]

In [7]:
pc = {(p,c) for p, c in df_kb[['STRIP_PEPTIDE', 'CHARGE']].values}
df_ided_peptide = pd.DataFrame([[c, p, pc2prot[(p, c)], pc2cnt[(p,c)], pc2mz[(p,c)], pc2int[(p,c)]]
                                for p, c in pc],
                              columns=['CHARGE', 'PEPTIDE', 'PROTEINS', 
                                       'SPECTRAL_CNT', 'MZ', 'INTENSITY'])

In [8]:
print(df_ided_peptide.shape)
df_ided_peptide.head(2)

(1457872, 6)


,CHARGE,PEPTIDE,PROTEINS,SPECTRAL_CNT,MZ,INTENSITY
0,3,LQMDNELLRQQIEELK,sp|Q15853|USF2_HUMAN,10,101.07118225097656;102.05537414550781;104.0531...,22109.3671875;8687.90234375;13804.5576171875;1...
1,2,VKHEDQQQGEDEH,sp|P05814|CASB_HUMAN,1,110.07174682617188;129.10267639160156;138.0550...,39044.734375;24020.220703125;23875.279296875;5...


# protein selection
  - only use ided protein (based on massIVE-KB)

In [9]:
ided_protein = list({__ for _ in df_ided_peptide.PROTEINS.values for __ in _.split(';')})
print(len(ided_protein))

21461


In [10]:
df_ided_protein_idx = [idx for idx, p in zip(df_uni.index, df_uni.PROTEIN.values) if p in ided_protein]
df_ided_protein = df_uni.loc[df_ided_protein_idx].reset_index(drop=True)
print(df_ided_protein.shape)
df_ided_protein.head(1)
# pk = PROTEIN

(19291, 3)


,PROTEIN,SEQUENCE,PROTEIN_FULL
0,sp|Q9H553|ALG2_HUMAN,MAEEQGRERDSVPKPSVLFLHPDLGVGGAERLVLDAALALQARGCS...,">sp|Q9H553|ALG2_HUMAN Alpha-1,3/1,6-mannosyltr..."


# Spectral Count
  - on uniprot protiens
  - with massIVE-KB peptides,
  - about spectral_count, spectral_count_miss

In [11]:
def prot2counting(pep_tree, df_prot, df_pep):
    # STRIP_PEPTIDE SPECTRAL_CNT
    pep2cnt = dict()
    for p, cnt in df_pep[['PEPTIDE', 'SPECTRAL_CNT']].values:
        if p not in pep2cnt:
            pep2cnt[p] = 0
        pep2cnt[p] += cnt

    # counting array init
    pep_cnt = [[0 for __ in range(len(_))] for _ in df_prot.SEQUENCE.values]
    miss_cnt = [[0 for __ in range(len(_))] for _ in df_prot.SEQUENCE.values]
    
    # counting start! (protein roop)
    for prot_idx, prot_seq in enumerate(df_prot.SEQUENCE.values):
        if prot_idx % 1000 == 0:
            print('peptide(spectral) counting...', prot_idx, '/', len(df_prot), end='\r')

        # matched peptide in protein
        results = pep_tree.search_all(prot_seq)
        # ex) result = ('ACAC', 5) = (peptide, location)
        for result in results: 
            pep = result[0]
            pep_len = len(result[0])
            start_idx = result[1]
            cnt = pep2cnt[pep]
            
            # miss cleavage case
            if sum([1 for p in pep[:-1] if p in 'KR']) >= 1:
                if pep[-1] in 'KR':  # __KR__KR case
                    miss_change_range = range(start_idx, start_idx + pep_len -1)
                    # miss cleavage count
                    for idx, value in enumerate(miss_cnt[prot_idx]):
                        if idx in miss_change_range:
                            value += cnt
                            miss_cnt[prot_idx][idx] = value
                    # peptide(spectral) count 
                    # only count first, and last amino acid of peptide which include miss cleavage
                    # last : For counting N term of tryptic site
                    # first : Foir counting C term of tryptic site
                    pep_cnt[prot_idx][start_idx + pep_len -1] += cnt
                    pep_cnt[prot_idx][start_idx] += cnt
                else:  # _KR__ case
                    miss_change_range = range(start_idx, start_idx + pep_len)
                    # miss cleavage count
                    for idx, value in enumerate(miss_cnt[prot_idx]):
                        if idx in miss_change_range:
                            value += cnt
                            miss_cnt[prot_idx][idx] = value
            # not miss cleavage case
            else:
                change_range = range(start_idx, start_idx + pep_len)
                # peptide(spectral) count
                for idx, value in enumerate(pep_cnt[prot_idx]):
                    if idx in change_range:
                        value += cnt
                        pep_cnt[prot_idx][idx] = value
    print('peptide(spectral) counting done!                              n')
    
    # peptide(spectral) count with protein
    prot2cnt = dict()
    for idx, ((pname, seqs), cnts) in enumerate(zip(df_prot[['PROTEIN', 'SEQUENCE']].values, pep_cnt)):
        prot2cnt[pname]=';'.join([str(seq) + '_' + str(cnt) for seq, cnt in zip(seqs, cnts)])

    prot2cnt_miss = dict()
    for idx, ((pname, seqs), cnts) in enumerate(zip(df_prot[['PROTEIN', 'SEQUENCE']].values, miss_cnt)):
        prot2cnt_miss[pname] = ';'.join([str(seq) + '_' + str(cnt) for seq, cnt in zip(seqs, cnts)])
    return prot2cnt, prot2cnt_miss

In [12]:
pep_tree = tree_from_pep(df_ided_peptide.PEPTIDE.unique())  # from utils

prot2cnt, prot2cnt_miss = prot2counting(pep_tree, df_ided_protein, df_ided_peptide)

df_ided_protein['SPECTRAL_CNT'] = [prot2cnt[pname] for pname in df_ided_protein.PROTEIN.values]
df_ided_protein['SPECTRAL_CNT_MISS'] = [prot2cnt_miss[pname] for pname in df_ided_protein.PROTEIN.values]

peptide(spectral) counting done!                              n


In [13]:
print(df_ided_protein.shape)
df_ided_protein.head(1)

(19291, 5)


,PROTEIN,SEQUENCE,PROTEIN_FULL,SPECTRAL_CNT,SPECTRAL_CNT_MISS
0,sp|Q9H553|ALG2_HUMAN,MAEEQGRERDSVPKPSVLFLHPDLGVGGAERLVLDAALALQARGCS...,">sp|Q9H553|ALG2_HUMAN Alpha-1,3/1,6-mannosyltr...",M_0;A_0;E_0;E_0;Q_0;G_0;R_0;E_3;R_0;D_76;S_2;V...,M_0;A_0;E_0;E_0;Q_0;G_0;R_0;E_3;R_3;D_79;S_81;...


# Make Detect_pep, unDetect_pep version==whole
  - spectral cnt >= 2 : detect
  - spectral cnt == 0 : undetect

In [14]:
def pep_from_prot(df_uni, MISS_CLEAVAGE, thres=4):
    peptide = []
    TS_AA = 'KR'

    for protein, pname, cnt, cnt_miss in df_uni[['SEQUENCE', 'PROTEIN', 'SPECTRAL_CNT', 'SPECTRAL_CNT_MISS']].values:
        cnt = list(map(lambda x: int(x[2:]), cnt.split(';')))
        cnt_miss = list(map(lambda x: int(x[2:]), cnt_miss.split(';')))
        
        ts_idx = []  # tryptic site index
        for prot_idx, aa in enumerate(protein):
            if aa in TS_AA:
                ts_idx.append(prot_idx)
        for idx in range(len(ts_idx)):
            n = MISS_CLEAVAGE
            if idx+(n+1) > len(ts_idx)-1:  # peptide making range
                break

            # protein N term
            if (ts_idx[idx]<=(thres - 1)) and (ts_idx[idx] >= len(protein) -1 -(thres - 1)):  # -MNQKLLK- 앞뒤 다 부족한 경우
                en = 'Z' * (thres - ts_idx[idx]) + protein[: ts_idx[idx]+(thres + 1)] + 'Z' * (thres - (len(protein)-1 - ts_idx[idx]))
            elif ts_idx[idx]<=(thres - 1):
                en = 'Z' * (thres - ts_idx[idx]) + protein[: ts_idx[idx]+(thres + 1)]
            elif ts_idx[idx] >= len(protein) -1 -(thres - 1):  # for EAQDRRN : 끝이 부족한 경우
                en = protein[ts_idx[idx]-thres:] + 'Z' * (thres - (len(protein)-1 - ts_idx[idx]))
            else:
                en = protein[ts_idx[idx]-thres: ts_idx[idx]+(thres + 1)]

            # protein C term
            if (ts_idx[idx+n+1] >= len(protein) -1 -(thres - 1)) and (ts_idx[idx+n+1] <= (thres - 1)):
                ec = 'Z' * (thres - ts_idx[idx+n+1]) + protein[: ts_idx[idx+n+1]+(thres + 1)]  + 'Z' * (thres - (len(protein)-1 - ts_idx[idx+n+1]))
            elif ts_idx[idx+n+1] >= len(protein) -1 -(thres - 1):
                ec = protein[ts_idx[idx+n+1] - thres :] + 'Z' * (thres - (len(protein)-1 - ts_idx[idx+n+1]))
            elif ts_idx[idx+n+1] <= (thres - 1):  # for -MRRS : 시작이 부족한 경우
                ec = 'Z' * (thres - ts_idx[idx+n+1]) + protein[: ts_idx[idx+n+1]+(thres + 1)]
            else:
                ec = protein[ts_idx[idx+n+1] - thres : ts_idx[idx+n+1] +(thres + 1)]  # n+1번째 tryptic_site
                    
            # Peptide miss cleavage
            if n != 0:  # miss cleavage 고려하는 경우,
                ei = []
                for i in range(1, n+1):
                    if ts_idx[idx+i] >= len(protein) -1 -(thres - 1):
                        ei_tmp = protein[ts_idx[idx+i] - thres :] + 'Z' * (thres - (len(protein)-1 - ts_idx[idx+i]))
                    elif ts_idx[idx+i] <= (thres - 1):
                        ei_tmp = 'Z' * (thres - ts_idx[idx+i]) + protein[: ts_idx[idx+i]+(thres + 1)]
                    else:
                        ei_tmp = protein[ts_idx[idx+i] - thres : ts_idx[idx+i] +(thres + 1)]
                    ei.append(ei_tmp)
            else:  # miss cleavage 고려안하는 경우
                ei = []
            # peptide
            if ts_idx[idx+n+1] == len(protein)-1:  # protein C term = idx + n + 1 의 tryptic_site 가 단백질의 마지막인 경우
                pep = protein[ts_idx[idx]] + '.' + protein[ts_idx[idx]+1:ts_idx[idx+n+1]+1] + '.Z'
            else:
                pep = protein[ts_idx[idx]] + '.' + protein[ts_idx[idx]+1:ts_idx[idx+n+1]+1] + '.' + protein[ts_idx[idx+n+1]+1]

            peptide.append({pep:[[en], [ec], ei, pname]})
    return peptide

In [15]:
# STRIP_PEPTIDE SPECTRAL_CNT
pep2cnt = dict()
for p, cnt in df_ided_peptide[['PEPTIDE', 'SPECTRAL_CNT']].values:
    if p not in pep2cnt:
        pep2cnt[p] = 0
    pep2cnt[p] += cnt
    
    
def pep_detection_labelling(df_ided_protein=df_ided_protein, pep_tree=pep_tree, pep2cnt=pep2cnt):
    peptide0 = pep_from_prot(df_ided_protein, 0, thres=7)
    peps = [list(dic.keys())[0] for dic in peptide0]
    ens = [list(dic.values())[0][0] for dic in peptide0]
    ecs = [list(dic.values())[0][1] for dic in peptide0]
    eis = [list(dic.values())[0][2] for dic in peptide0]
    pname = [list(dic.values())[0][3] for dic in peptide0]
    zero = pd.DataFrame([[p, n[0], c[0], '-', '-', pn] 
                         for p, n, c, m, pn in zip(peps, ens, ecs, eis, pname)],
                columns=['peptide', 'En', 'Ec', 'E1', 'E2', 'protein'])
    zero['PEP'] = [i.split('.')[1] for i in zero.peptide.values]

    peptide1 = pep_from_prot(df_ided_protein, 1, thres=7)
    peps = [list(dic.keys())[0] for dic in peptide1]
    ens = [list(dic.values())[0][0] for dic in peptide1]
    ecs = [list(dic.values())[0][1] for dic in peptide1]
    eis = [list(dic.values())[0][2] for dic in peptide1]
    pname = [list(dic.values())[0][3] for dic in peptide1]
    one = pd.DataFrame([[p, n[0], c[0], m[0], '-', pn]
                        for p, n, c, m, pn in zip(peps, ens, ecs, eis, pname)],
                columns=['peptide', 'En', 'Ec', 'E1', 'E2', 'protein'])
    one['PEP'] = [i.split('.')[1] for i in one.peptide.values]

    peptide2 = pep_from_prot(df_ided_protein, 2, thres=7)
    peps = [list(dic.keys())[0] for dic in peptide2]
    ens = [list(dic.values())[0][0] for dic in peptide2]
    ecs = [list(dic.values())[0][1] for dic in peptide2]
    eis = [list(dic.values())[0][2] for dic in peptide2]
    pname = [list(dic.values())[0][3] for dic in peptide2]
    two = pd.DataFrame([[p, n[0], c[0], m[0], m[1], pn]
                        for p, n, c, m, pn in zip(peps, ens, ecs, eis, pname)],
                columns=['peptide', 'En', 'Ec', 'E1', 'E2', 'protein'])
    two['PEP'] = [i.split('.')[1] for i in two.peptide.values]

    zero_filter = zero.loc[zero.PEP.apply(lambda x: (len(x)>=7) and (len(x)<=30))]
    one_filter = one.loc[one.PEP.apply(lambda x: (len(x)>=7) and (len(x)<=30))]
    two_filter = two.loc[two.PEP.apply(lambda x: (len(x)>=7) and (len(x)<=30))]

    df_fully = pd.concat([zero_filter, one_filter, two_filter], axis=0).reset_index(drop=True)

    # preprocessing
    amino = list("ARNDCEQGHILKMFPSTWYVZ")
    check_u = [idx for idx, t in enumerate(df_fully.PEP.values) if 'U' in t]
    check_x = [idx for idx, t in enumerate(df_fully.PEP.values) if 'X' in t]
    check_n = [idx for idx, t in enumerate(df_fully.En.values) if (sum([1 for a in t if a not in amino]) >= 1) and t !='-']
    check_c = [idx for idx, t in enumerate(df_fully.Ec.values) if (sum([1 for a in t if a not in amino]) >= 1) and t !='-']
    check_1 = [idx for idx, t in enumerate(df_fully.E1.values) if (sum([1 for a in t if a not in amino]) >= 1) and t !='-']
    check_2 = [idx for idx, t in enumerate(df_fully.E2.values) if (sum([1 for a in t if a not in amino]) >= 1) and t !='-']
    check = list(set(check_n).union(set(check_c)).union(set(check_1)).union(set(check_2)).union(set(check_u)).union(set(check_x)))
    print('del rows(including U, X amino acid) cnt : {:,}'.format(len(check)))
    df_fully = df_fully.drop(check).reset_index(drop=True)

    ided = []
    for fully_pep in df_fully.PEP.values:
        results = [i for i in pep_tree.search_all(fully_pep)]
        if len(results) != 0:
            result_pep = [_[0] for _ in results]
            if fully_pep in result_pep:  # 정확히 똑같은 pep 있는 경우
                if pep2cnt[fully_pep] > 1:
                    ided.append(1)
                else:
                    ided.append(-1)  # SC1인 경우는 제거하자
            else:
                ided.append(0)
        else:
            ided.append(0)
    df_fully['ID'] = ided

    drop_idx = df_fully.loc[df_fully.ID==-1].index
    print('peptide with 1 spectral count (remove this peptide) cnt : ', len(drop_idx))
    df_fully = df_fully.drop(drop_idx).reset_index(drop=True)

    drop_idx = np.where(df_fully.duplicated()==True)[0]
    df_fully = df_fully.drop(drop_idx).reset_index(drop=True)

    print('----ID/nonID cnt----\n', df_fully.ID.value_counts())  # massIVE-KB : half is semi_non_tryptic, half is fully_tryptic
    print('----final shape----\n', df_fully.shape)
    df_fully.head(1)
    return df_fully

In [16]:
df_kb.loc[df_kb.STRIP_PEPTIDE=='AAEEPSKVEEK']

,Unnamed: 0,PEPMASS,CHARGE,FILENAME,SEQ,PROTEIN,STRIP_PEPTIDE,SCORE,MZ,INTENSITY,SPECTRAL_CNT,PROTEIN_except_decoy,PROTEIN_except_else,PTM_SEQ
1,1,420.207611,3,filtered_library_mgf_files/9187af535f6e41ce958...,+42.011AAEEPSKVEEK,sp|P29966|MARCS_HUMAN,AAEEPSKVEEK,5.081104,102.05455017089844;111.09056091308594;129.1016...,6125.9912109375;1650.5614013671875;13222.81445...,3,sp|P29966|MARCS_HUMAN,sp|P29966|MARCS_HUMAN,(Ace)AAEEPSKVEEK
79983,79983,629.808777,2,filtered_library_mgf_files/4ceb8327858446e3b58...,+42.011AAEEPSKVEEK,sp|P29966|MARCS_HUMAN,AAEEPSKVEEK,4.603983,102.05474090576172;111.09119415283203;115.0859...,1351.0970458984375;856.5582885742188;587.15649...,7,sp|P29966|MARCS_HUMAN,sp|P29966|MARCS_HUMAN,(Ace)AAEEPSKVEEK
586338,586338,608.806947,2,filtered_library_mgf_files/3874413ebf874cf19c9...,AAEEPSKVEEK,sp|P29966|MARCS_HUMAN,AAEEPSKVEEK,5.561619,102.05512237548828;115.08656311035156;115.0940...,144564.03125;309713.1875;10859.046875;110717.7...,95,sp|P29966|MARCS_HUMAN,sp|P29966|MARCS_HUMAN,AAEEPSKVEEK
732863,732863,406.208155,3,filtered_library_mgf_files/0c118258c5c04954969...,AAEEPSKVEEK,sp|P29966|MARCS_HUMAN,AAEEPSKVEEK,5.561619,102.05590057373047;110.07201385498047;115.0874...,146919.9375;38942.06640625;651668.3125;25144.4...,85,sp|P29966|MARCS_HUMAN,sp|P29966|MARCS_HUMAN,AAEEPSKVEEK
1938537,1938537,630.309520,2,filtered_library_mgf_files/f9dc38f66afa49a2b0a...,+43.006AAEEPSKVEEK,sp|P29966|MARCS_HUMAN,AAEEPSKVEEK,5.384195,87.055908203125;102.05543518066406;110.0719375...,23255.646484375;65401.15234375;5587.6552734375...,84,sp|P29966|MARCS_HUMAN,sp|P29966|MARCS_HUMAN,(Cab)AAEEPSKVEEK


In [17]:
pep2cnt['AAEEPSKVEEK']

2

In [18]:
df_fully = pep_detection_labelling()

del rows(including U, X amino acid) cnt : 160
peptide with 1 spectral count (remove this peptide) cnt :  113427
----ID/nonID cnt----
 0    1493757
1     480223
Name: ID, dtype: int64
----final shape----
 (1973980, 8)


In [19]:
nidp = df_fully.loc[df_fully.ID==0].PEP.unique()
idp = df_fully.loc[df_fully.ID==1].PEP.unique()
print(len(set(nidp).intersection(set(idp))))

0


In [20]:
data = []
for prot, ptms, strips, c in df_kb[['PROTEIN', 'PTM_SEQ', 'STRIP_PEPTIDE', 'CHARGE']].values:
    for p in prot.split(';'):
        data.append([p, ptms, strips, c])
df_key = pd.DataFrame(data, columns=['protein', 'PTM', 'PEP', 'CHARGE']).drop_duplicates().reset_index(drop=True)
print(df_key.shape, len({(_, __) for _, __ in df_key[['protein', 'PEP']].values}))

df_fully_ptms = df_fully.loc[df_fully.ID==1].merge(df_key, on=['protein', 'PEP'], how='left')
print(df_fully_ptms.shape)

(2189501, 4) 1121725
(1244846, 10)


In [21]:
df_fully_ptms.head(2)

,peptide,En,Ec,E1,E2,protein,PEP,ID,PTM,CHARGE
0,K.PSVLFLHPDLGVGGAER.L,RERDSVPKPSVLFLH,LGVGGAERLVLDAAL,-,-,sp|Q9H553|ALG2_HUMAN,PSVLFLHPDLGVGGAER,1,PSVLFLHPDLGVGGAER,2.0
1,R.LVLDAALALQAR.G,LGVGGAERLVLDAAL,AALALQARGCSVKIW,-,-,sp|Q9H553|ALG2_HUMAN,LVLDAALALQAR,1,LVLDAALALQAR,2.0


In [22]:
df_fully_ptms.loc[df_fully_ptms.PTM.isnull()].head(2)

,peptide,En,Ec,E1,E2,protein,PEP,ID,PTM,CHARGE
8682,R.LLLAALVQDYVQMK.A,TLSEDEARLLLAALV,VQDYVQMKASELEQE,-,-,sp|P06881|CALCA_HUMAN,LLLAALVQDYVQMK,1,NaN,NaN
168078,R.LLLAALVQDYVQMK.A,TLSEDEARLLLAALV,VQDYVQMKASELEQE,-,-,sp|P01258|CALC_HUMAN,LLLAALVQDYVQMK,1,NaN,NaN


In [23]:
df_fully_ptms.loc[df_fully_ptms.PEP=='LLLAALVQDYVQMK']

,peptide,En,Ec,E1,E2,protein,PEP,ID,PTM,CHARGE
8682,R.LLLAALVQDYVQMK.A,TLSEDEARLLLAALV,VQDYVQMKASELEQE,-,-,sp|P06881|CALCA_HUMAN,LLLAALVQDYVQMK,1,NaN,NaN
168078,R.LLLAALVQDYVQMK.A,TLSEDEARLLLAALV,VQDYVQMKASELEQE,-,-,sp|P01258|CALC_HUMAN,LLLAALVQDYVQMK,1,NaN,NaN
183061,R.LLLAALVQDYVQMK.A,TLSKEDARLLLAALV,VQDYVQMKASELKQE,-,-,sp|P10092|CALCB_HUMAN,LLLAALVQDYVQMK,1,LLLAALVQDYVQM(Ox)K,2.0


In [24]:
df_fully_ptms.isnull().sum()

peptide     0
En          0
Ec          0
E1          0
E2          0
protein     0
PEP         0
ID          0
PTM        36
CHARGE     36
dtype: int64

In [25]:
df_fully_ptms = df_fully_ptms.dropna().reset_index(drop=True)

In [26]:
cab_idx = [idx for idx, seq in zip(df_fully_ptms.index, df_fully_ptms.PTM.values) if '(Cab)' in seq]
ace_idx = [idx for idx, seq in zip(df_fully_ptms.index, df_fully_ptms.PTM.values) if '(Ace)' in seq]
aml_idx = [idx for idx, seq in zip(df_fully_ptms.index, df_fully_ptms.PTM.values) if '(Aml)' in seq]
drop_idx = set(cab_idx).union(set(ace_idx)).union(set(aml_idx))
df_fully_ptms=df_fully_ptms.drop(drop_idx, axis=0).reset_index(drop=True)

In [27]:
len(drop_idx)

147737

In [28]:
print(df_fully_ptms.shape)
df_fully_ptms.head(2)

(1097073, 10)


,peptide,En,Ec,E1,E2,protein,PEP,ID,PTM,CHARGE
0,K.PSVLFLHPDLGVGGAER.L,RERDSVPKPSVLFLH,LGVGGAERLVLDAAL,-,-,sp|Q9H553|ALG2_HUMAN,PSVLFLHPDLGVGGAER,1,PSVLFLHPDLGVGGAER,2.0
1,R.LVLDAALALQAR.G,LGVGGAERLVLDAAL,AALALQARGCSVKIW,-,-,sp|Q9H553|ALG2_HUMAN,LVLDAALALQAR,1,LVLDAALALQAR,2.0


In [29]:
detect_peps = df_fully_ptms.PTM.apply(lambda x: x.replace('(Cabm)', '').replace('(Dam)', '').replace('(Ox)', '')).values
detect_charge = df_fully_ptms.CHARGE.values

In [30]:
detect_pc = {(p, c) for p, c in zip(detect_peps, detect_charge)}

In [31]:
print(len(detect_pc))

764321


In [32]:
df_kb.head(1)

,Unnamed: 0,PEPMASS,CHARGE,FILENAME,SEQ,PROTEIN,STRIP_PEPTIDE,SCORE,MZ,INTENSITY,SPECTRAL_CNT,PROTEIN_except_decoy,PROTEIN_except_else,PTM_SEQ
0,0,623.327626,3,filtered_library_mgf_files/9187af535f6e41ce958...,+42.011AAEAAGGKYRSTVSKSKD,sp|Q96A65|EXOC4_HUMAN,AAEAAGGKYRSTVSKSKD,4.605175,129.10317993164062;130.08726501464844;136.0766...,63295.375;7010.876953125;14278.6826171875;5026...,1,sp|Q96A65|EXOC4_HUMAN,sp|Q96A65|EXOC4_HUMAN,(Ace)AAEAAGGKYRSTVSKSKD


In [33]:
len({(p, c) for p, c in df_kb[['STRIP_PEPTIDE', 'CHARGE']].values}), len(df_kb)

(1457872, 1991465)

In [34]:
df_key_realspec = pd.DataFrame([[p, c, '-'] for p, c in detect_pc], columns=['STRIP_PEPTIDE', 'CHARGE', 'DROP'])
df_kbreal = df_kb.merge(df_key_realspec, on=['STRIP_PEPTIDE', 'CHARGE'], how='inner')

In [35]:
cab_idx = [idx for idx, seq in zip(df_kbreal.index, df_kbreal.PTM_SEQ.values) if '(Cab)' in seq]
ace_idx = [idx for idx, seq in zip(df_kbreal.index, df_kbreal.PTM_SEQ.values) if '(Ace)' in seq]
aml_idx = [idx for idx, seq in zip(df_kbreal.index, df_kbreal.PTM_SEQ.values) if '(Aml)' in seq]
drop_idx = set(cab_idx).union(set(ace_idx)).union(set(aml_idx))
df_kbreal=df_kbreal.drop(drop_idx, axis=0).reset_index(drop=True)

In [36]:
df_kbreal.shape

(1035396, 15)

In [37]:
len({(p, c) for p, c in df_kbreal[['STRIP_PEPTIDE', 'CHARGE']].values})

764321

In [38]:
len({(p, c) for p, c in df_kbreal[['PTM_SEQ', 'CHARGE']].values})

1035396

# guan Charge prediction
# prosit MS2 prediction

In [39]:
import tensorflow as tf

In [40]:
import pickle
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers

import matplotlib.pyplot as plt
from scipy.stats import pearsonr

import pandas as pd

In [41]:
def reshapeOneHot(X):
    X = np.dstack(X)
    X = np.swapaxes(X, 1, 2)
    X = np.swapaxes(X, 0, 1)
    return X

def one_hot_encode_peptide(peptide, MAX_LENGTH = 41):
    if len(peptide) > MAX_LENGTH:
        print('Peptide length is larger than maximal length of ', str(MAX_LENGTH))
        return None
    else:
        AA_vocabulary = 'KRPTNAQVSGILCMJHFYWEDBXO'#B: acetyl; O: Carbamyl; J: oxidized Met; X:pyro_glu
        no_not_used_aas = 1#U: not used

        one_hot_peptide = np.zeros((len(peptide), len(AA_vocabulary) - no_not_used_aas))

        for j in range(0, len(peptide)):
            try:
                aa = peptide[j]
                one_hot_peptide[j, AA_vocabulary.index(aa)] = 1
            except:
                pass
        
        no_front_paddings = int((MAX_LENGTH - len(peptide))/2)
        peptide_front_paddings = np.zeros((no_front_paddings, one_hot_peptide.shape[1]))

        no_back_paddings = MAX_LENGTH - len(peptide) - no_front_paddings
        peptide_back_paddings = np.zeros((no_back_paddings, one_hot_peptide.shape[1]))

        full_one_hot_peptide = np.vstack((peptide_front_paddings, one_hot_peptide, peptide_back_paddings))

        return peptide, full_one_hot_peptide
    
AA_vocabulary = 'KRPTNAQVSGILCMJHFYWEDBXOU'#B: acetyl; J: oxidized Met; X:pyro_glu
single2psi_lookup = {'B': '(Acetyl)-',
                     'J': 'M(Oxidation)',
                     'X': '(Gln->pyro-Glu)Q',
                     'C': 'C(Carbamidomethyl)'}

def one_hot_to_single_ptm(one_hot):
    seq = ''
    for row in one_hot:
        if row.sum() > 0:
            idx = row.argmax()
            seq += AA_vocabulary[idx]
    return seq



def single_ptm_to_psi(single_ptm_seq):
    psi_seq = single_ptm_seq
    for code in single2psi_lookup:
        psi_seq = psi_seq.replace(code, single2psi_lookup[code])
    return psi_seq

fn1 = '../refModel/2019MCP_Guan/LCMSMS_Pred_Supplemental_Material_section_S6/ChargeState/zfit_bidirLSTM2_masking_result.pickle'
with open(fn1, 'rb') as fid:
    pred_data = pickle.load(fid)

fn2 = '../refModel/2019MCP_Guan/LCMSMS_Pred_Supplemental_Material_section_S6/ChargeState/zfit_one_hot.pickle'
with open(fn2, 'rb') as fid:
    real_data = pickle.load(fid)

def plot_zfit(train_X, train_y, predicted_train_y, n):

    X = train_X[n]
    s_seq = one_hot_to_single_ptm(X)
    print(s_seq)
    psi_seq = single_ptm_to_psi(s_seq)
    print(psi_seq)
   
    y_exp = train_y[n]
    y_pred = predicted_train_y[n]
    pcc = pearsonr(y_exp, y_pred)

    fig, ax = plt.subplots()
    charges = range(1, len(y_exp) + 1)
    ax.stem(charges, y_exp, 'b', markerfmt=" ", label='Experimental')
    ax.stem(charges, -y_pred, 'g', markerfmt=" ", label='Predicted')
    plt.text(1.5, -0.35, 'pcc: %6.3f, %6.3g'%(pcc[0], pcc[1]))
    plt.xticks(charges)
    plt.xlabel('charge')
    plt.title(psi_seq)
    plt.show()

In [42]:
network_ori = tf.keras.models.load_model('../refModel/2019MCP_Guan/LCMSMS_Pred_Supplemental_Material_section_S6/ChargeState/zfit_bidirLSTM2_masking_model.h5')

print(network_ori.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, None, 23)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 512)         573440    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               1574912   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1285      
Total params: 2,280,965
Trainable params: 2,280,965
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
undetect_peps = df_fully.loc[df_fully.ID==0].PEP.unique()
print(len(undetect_peps))  # 교집합 없음

1422466


* undetected peptides

In [44]:
# additional prediction charge 6 peptide 
all_X_str = np.array(list(set(undetect_peps)))
all_onehot = dict()
for p in all_X_str:
    o = one_hot_encode_peptide(p)
    all_onehot[p] = o[1]
all_X = np.array(list(all_onehot.values()))

In [45]:
print(all_X.shape)

(1422466, 41, 23)


In [46]:
test=all_X[:1000]
kb_pred = network_ori.predict(test)

In [47]:
print(round(16.4*(1422466/1000)/3600, 2), 'hour estimated')

6.48 hour estimated


In [48]:
import math

In [49]:
all_pred = network_ori.predict(all_X)

In [50]:
all_pred_log = np.array([np.array(list(map(lambda x: -math.log(x), _))) for _ in all_pred])
print(all_X_str.shape, all_X.shape, all_pred_log.shape)

peplist = []
for p, prob in zip(all_X_str, all_pred_log):
    idxes = np.where(prob<=6)[0]
    charges = idxes + 1  # casting
    for c in charges:
        peplist.append([p, 30, c])
df_peplist = pd.DataFrame(peplist, columns=['modified_sequence', 'collision_energy', 'precursor_charge'])

(1422466,) (1422466, 41, 23) (1422466, 5)


In [51]:
# # 7시간 예상.. 일단 charge 2로 통일
# df_peplist = pd.DataFrame([[_, 30, 2] for _ in undetect_peps], columns=['modified_sequence', 'collision_energy', 'precursor_charge'])

In [52]:
print(df_peplist.shape)

(4349368, 3)


In [53]:
df_peplist.to_csv('undetect_peptides_MS2prediction_211115.csv', index=False)

* detected peptide

In [56]:
detect_peps = [_[0] for _ in detect_pc]

In [57]:
# additional prediction charge 6 peptide 
all_X_str = np.array(list(set(undetect_peps)))
all_onehot = dict()
for p in all_X_str:
    o = one_hot_encode_peptide(p)
    all_onehot[p] = o[1]
all_X = np.array(list(all_onehot.values()))

In [58]:
print(all_X.shape)

(1422466, 41, 23)


In [59]:
test=all_X[:1000]
kb_pred = network_ori.predict(test)

In [60]:
print(round(14.5*(1422466/1000)/3600, 2), 'hour estimated')

5.73 hour estimated


In [ ]:
all_pred = network_ori.predict(all_X)

In [ ]:
all_pred_log = np.array([np.array(list(map(lambda x: -math.log(x), _))) for _ in all_pred])
print(all_X_str.shape, all_X.shape, all_pred_log.shape)

peplist = []
for p, prob in zip(all_X_str, all_pred_log):
    idxes = np.where(prob<=6)[0]
    charges = idxes + 1  # casting
    for c in charges:
        peplist.append([p, 30, c])
df_peplist = pd.DataFrame(peplist, columns=['modified_sequence', 'collision_energy', 'precursor_charge'])

In [ ]:
print(df_peplist.shape)

In [ ]:
df_peplist.to_csv('detect_peptides_MS2prediction_211115.csv', index=False)

# prosit to MGF
# MSGF search

In [53]:
undetect_ms2 = pd.read_csv('undetect_peptides_MS2prediction_211025_result.msms', sep='\t')

In [54]:
print(undetect_ms2.shape)
undetect_ms2.head(2)

(1422466, 5)


,Intensities,Masses,Matches,Modified Sequence,Charge
0,0.27977493;0.91897136;0.45183122;0.5894868;0.4...,147.112804167;294.18121816699994;393.249632166...,y1;y2;y3;y4;y5;y6;y7;y8;y9;y10;y11;y12;y13;y14...,APIDWIEEYTTGMADCILVNSQFTAAVFK,2
1,0.19809067;0.2571014;0.12245454;0.28557485;0.1...,175.118952167;246.156066167;345.22448016699997...,y1;y2;y3;y4;y5;y6;y7;y8;y9;y10;y11;y12;y13;y14...,LQEPEEDEDCYILNVQSSSDDTSGSSVAR,2


In [55]:
[_ for _ in undetect_ms2['Modified Sequence'] if 'X' in _]

[]

In [56]:
# Amino acids
MODIFICATION = {
    "CAM": 57.0214637236,  # Carbamidomethylation (CAM)
    "OX": 15.99491,  # Oxidation
}
AMINO_ACID = {
    "G": 57.021464,
    "R": 156.101111,
    "V": 99.068414,
    "P": 97.052764,
    "S": 87.032028,
    "U": 150.95363,
    "L": 113.084064,
    "M": 131.040485,
    "Q": 128.058578,
    "N": 114.042927,
    "Y": 163.063329,
    "E": 129.042593,
    "C": 103.009185,
    "F": 147.068414,
    "I": 113.084064,
    "A": 71.037114,
    "T": 101.047679,
    "W": 186.079313,
    "H": 137.058912,
    "D": 115.026943,
    "K": 128.094963,
}
AMINO_ACID["M(ox)"] = AMINO_ACID["M"] + MODIFICATION["OX"]
AMINO_ACID["C(cam)"] = AMINO_ACID["C"] + MODIFICATION["CAM"]

# Atomic elements
PROTON = 1.007276467
ELECTRON = 0.00054858
H = 1.007825035
C = 12.0
O = 15.99491463
N = 14.003074

# Tiny molecules
N_TERMINUS = H
C_TERMINUS = O + H
CO = C + O
CHO = C + H + O
NH2 = N + H * 2
H2O = H * 2 + O
NH3 = N + H * 3

def cal_MW(seq):
    R = sum([AMINO_ACID[aa] for aa in list(map(lambda x: x.replace('C', 'C(cam)'), seq))])
    return R + N_TERMINUS + C_TERMINUS

def cal_mz(seq, CHARGE):
    # we treat STRIP_PEPTIDE, so modification does not consider.
    MW = cal_MW(seq)
    return (MW + PROTON*CHARGE) / CHARGE

In [57]:
import time

In [58]:
# peptide to spectrum dictionary (prediction)
start = time.time()

npep2spec_pred = dict()
for idx, (i, mz, p, c) in enumerate(undetect_ms2[['Intensities', 'Masses', 'Modified Sequence', 'Charge']].values):
    if idx % 100000 == 0:
        print(idx, round(time.time() - start, 2), end='\r')

    if (p, c) not in npep2spec_pred:
        npep2spec_pred[(p, c)] = dict()
    for i_, mz_ in zip(i.split(';'), mz.split(';')):
        npep2spec_pred[(p, c)][mz_]=i_

In [ ]:
cols = ['Intensities', 'Masses', 'Modified Sequence', 'Charge']
pep2cmzint = {p:[c, m, i] for i, m, p, c in df_ms2[cols].values}

get_idx = lambda x: int(round(float(x), 2)*100)

def get_spectrum(pep2cmzint):
    pep2tensor = dict()
    for pep, (c, m, i) in pep2cmzint.items():
        tensor = np.zeros((5, 100000))
        i = np.array(list(map(float, i.split(';'))))
        m2idx = np.array(list(map(get_idx, m.split(';'))))
        idx2inten = {k:v for k, v in zip(m2idx, i) if k <= 100000-1}  # Dalton 제한
        
        row_idx = c-1
        col_idx = np.array(list(idx2inten.keys()))
        val = np.array(list(idx2inten.values()))
        tensor[row_idx, col_idx] = val
        
        pep2tensor[pep]=tensor
    return pep2tensor

pep2tensor = get_spectrum(pep2cmzint)

In [59]:
# nID mgf
start = time.time()

idf01 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac01.mgf', mode='w')
idf02 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac02.mgf', mode='w')
idf03 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac03.mgf', mode='w')
idf04 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac04.mgf', mode='w')
idf05 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac05.mgf', mode='w')
idf06 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac06.mgf', mode='w')
idf07 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac07.mgf', mode='w')
idf08 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac08.mgf', mode='w')
idf09 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac09.mgf', mode='w')
idf10 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac10.mgf', mode='w')
idf11 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac11.mgf', mode='w')
idf12 = open('/data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac12.mgf', mode='w')
idf2frac = dict()
idf2frac[1]=idf01
idf2frac[2]=idf02
idf2frac[3]=idf03
idf2frac[4]=idf04
idf2frac[5]=idf05
idf2frac[6]=idf06
idf2frac[7]=idf07
idf2frac[8]=idf08
idf2frac[9]=idf09
idf2frac[10]=idf10
idf2frac[11]=idf11
idf2frac[12]=idf12
bins = [int(len(npep2spec_pred)/12 * i) for i in range(12)]
idx2fracidx = np.digitize(range(len(npep2spec_pred)), bins=bins)

for idx, ((p, c), mz2int) in enumerate(npep2spec_pred.items()):
    if idx % 100000 == 0:
        print(idx, round(time.time() - start, 2), end='\r')
        
    idf = idf2frac[idx2fracidx[idx]]
    idf.write('BEGIN IONS\n')
    idf.write('TITLE='+str(idx+1)+' File:\"uniprot_predMS2_frac{}.mgf", NativeID:\"controllerType=0 controllerNumber=1 scan='.format(str(idx+1).zfill(2))+str(idx+1)+'\"\n')
#     idf.write('RTINSECONDS=10\n')
    idf.write('PEPMASS='+str(cal_mz(p, c))+'\n')  # ' precursor_intensity'
    idf.write('CHARGE='+str(c)+'+\n')
    for mz, i in sorted(mz2int.items(), key=lambda x: x[0]):
        idf.write(str(mz)+' '+str(i)+'\n')
    idf.write('END IONS\n')
idf01.close()
idf02.close()
idf03.close()
idf04.close()
idf05.close()
idf06.close()
idf07.close()
idf08.close()
idf09.close()
idf10.close()
idf11.close()
idf12.close()

In [60]:
detect_ms2 = pd.read_csv('detect_peptides_MS2prediction_211025_result.msms', sep='\t')
print(detect_ms2.shape)
detect_ms2.head(2)

(764321, 5)


,Intensities,Masses,Matches,Modified Sequence,Charge
0,0.23332144;0.12056828;1.0;0.51005423;0.1888672...,175.118952167;288.203016167;375.235044167;474....,y1;y2;y3;y4;y5;y6;y7;y8;b1;b2;b3;b4;b5;b6;b7;y...,KKYEEEVSLR,3
1,0.1725661;0.33016136;0.10774748;0.2737293;0.60...,175.118952167;304.161545167;433.204138167;562....,y1;y2;y3;y4;y5;y6;y7;y8;b2;b3;b4;b5;b6;b7;b8;b...,AGVLAHLEEER,2


In [61]:
[_ for _ in detect_ms2['Modified Sequence'] if 'X' in _]

[]

In [62]:
[_ for _ in detect_ms2['Modified Sequence'] if 'M(ox)' in _]

[]

# massIVE-KB spectra

In [98]:
detect_ms2.head(1)

,Intensities,Masses,Matches,Modified Sequence,Charge
0,0.23332144;0.12056828;1.0;0.51005423;0.1888672...,175.118952167;288.203016167;375.235044167;474....,y1;y2;y3;y4;y5;y6;y7;y8;b1;b2;b3;b4;b5;b6;b7;y...,KKYEEEVSLR,3


In [101]:
df_kbreal.head(2)

,Unnamed: 0,PEPMASS,CHARGE,FILENAME,SEQ,PROTEIN,STRIP_PEPTIDE,SCORE,MZ,INTENSITY,SPECTRAL_CNT,PROTEIN_except_decoy,PROTEIN_except_else,PTM_SEQ,DROP
0,732863,406.208155,3,filtered_library_mgf_files/0c118258c5c04954969...,AAEEPSKVEEK,sp|P29966|MARCS_HUMAN,AAEEPSKVEEK,5.561619,102.05590057373047;110.07201385498047;115.0874...,146919.9375;38942.06640625;651668.3125;25144.4...,85,sp|P29966|MARCS_HUMAN,sp|P29966|MARCS_HUMAN,AAEEPSKVEEK,-
1,121794,653.582825,4,filtered_library_mgf_files/97f2d7cfb5a94249ad0...,AC+57.021QDVSVIIHTAC+57.021IIDVFGVTHR,sp|P14060|3BHS1_HUMAN,ACQDVSVIIHTACIIDVFGVTHR,9.530160,101.071044921875;115.08629608154297;120.080528...,3950.259521484375;7356.15966796875;18369.31445...,10,sp|P14060|3BHS1_HUMAN,sp|P14060|3BHS1_HUMAN,AC(Cabm)QDVSVIIHTAC(Cabm)IIDVFGVTHR,-


In [138]:
df_kbreal.shape

(1035396, 15)

In [114]:
len({(pp, c) for pp, p, c in df_kbreal[['STRIP_PEPTIDE', 'PTM_SEQ', 'CHARGE']].values if '(Ox)' not in p and '(Dam)' not in p})

748884

In [132]:
detect_ms2real = df_kbreal.loc[df_kbreal.PTM_SEQ.apply(lambda x: '(Ox)' not in x) & df_kbreal.PTM_SEQ.apply(lambda x: '(Dam)' not in x)]
detect_ms2real = detect_ms2real[['INTENSITY', 'MZ', 'STRIP_PEPTIDE', 'CHARGE']].rename({'INTENSITY':'Intensities',
                                                                                       'MZ':'Masses',
                                                                                       'STRIP_PEPTIDE':'Modified Sequence',
                                                                                       'CHARGE':'Charge'}, axis=1)

In [134]:
# peptide to spectrum dictionary
start = time.time()

pep2spec = dict()
for idx, (i, mz, p, c) in enumerate(detect_ms2real[['Intensities', 'Masses', 'Modified Sequence', 'Charge']].values):
    if idx % 100000 == 0:
        print(idx, round(time.time() - start, 2), end='\r')

    if (p, c) not in pep2spec:
        pep2spec[(p, c)] = dict()
    for i_, mz_ in zip(i.split(';'), mz.split(';')):
        pep2spec[(p, c)][mz_]=i_

In [63]:
# peptide to spectrum dictionary (prediction)
start = time.time()

pep2spec_pred = dict()
for idx, (i, mz, p, c) in enumerate(detect_ms2[['Intensities', 'Masses', 'Modified Sequence', 'Charge']].values):
    if idx % 100000 == 0:
        print(idx, round(time.time() - start, 2), end='\r')

    if (p, c) not in pep2spec_pred:
        pep2spec_pred[(p, c)] = dict()
    for i_, mz_ in zip(i.split(';'), mz.split(';')):
        pep2spec_pred[(p, c)][mz_]=i_

In [64]:
# ID mgf
start = time.time()

idf01 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac01.mgf', mode='w')
idf02 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac02.mgf', mode='w')
idf03 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac03.mgf', mode='w')
idf04 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac04.mgf', mode='w')
idf05 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac05.mgf', mode='w')
idf06 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac06.mgf', mode='w')
idf07 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac07.mgf', mode='w')
idf08 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac08.mgf', mode='w')
idf09 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac09.mgf', mode='w')
idf10 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac10.mgf', mode='w')
idf11 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac11.mgf', mode='w')
idf12 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_predMS2_frac12.mgf', mode='w')
idf2frac = dict()
idf2frac[1]=idf01
idf2frac[2]=idf02
idf2frac[3]=idf03
idf2frac[4]=idf04
idf2frac[5]=idf05
idf2frac[6]=idf06
idf2frac[7]=idf07
idf2frac[8]=idf08
idf2frac[9]=idf09
idf2frac[10]=idf10
idf2frac[11]=idf11
idf2frac[12]=idf12
bins = [int(len(pep2spec_pred)/12 * i) for i in range(12)]
idx2fracidx = np.digitize(range(len(pep2spec_pred)), bins=bins)

for idx, ((p, c), mz2int) in enumerate(pep2spec_pred.items()):
    if idx % 100000 == 0:
        print(idx, round(time.time() - start, 2), end='\r')
    idf = idf2frac[idx2fracidx[idx]]
    idf.write('BEGIN IONS\n')
    idf.write('TITLE='+str(idx+1)+' File:\"massIVE-KB_predMS2_frac{}.mgf", NativeID:\"controllerType=0 controllerNumber=1 scan='.format(str(idx+1).zfill(2))+str(idx+1)+'\"\n')
    idf.write('PEPMASS='+str(cal_mz(p, c))+'\n')  # ' precursor_intensity'
    idf.write('CHARGE='+str(c)+'+\n')
    for mz, i in sorted(mz2int.items(), key=lambda x: x[0]):
        idf.write(str(mz)+' '+str(i)+'\n')
    idf.write('END IONS\n')

idf01.close()
idf02.close()
idf03.close()
idf04.close()
idf05.close()
idf06.close()
idf07.close()
idf08.close()
idf09.close()
idf10.close()
idf11.close()
idf12.close()

In [65]:
import pickle
with open('../data/massIVE-KB/211025_stirp_pep2spec_pred.pickle', 'wb') as f:
    pickle.dump(pep2spec_pred, f, pickle.HIGHEST_PROTOCOL)
    
with open('../data/uniprot/211025_strip_npep2spec_pred.pickle', 'wb') as f:
    pickle.dump(npep2spec_pred, f, pickle.HIGHEST_PROTOCOL)

In [135]:
with open('../data/massIVE-KB/211108_stirp_pep2spec.pickle', 'wb') as f:
    pickle.dump(pep2spec, f, pickle.HIGHEST_PROTOCOL)

In [137]:
# ID mgf
start = time.time()

idf01 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac01.mgf', mode='w')
idf02 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac02.mgf', mode='w')
idf03 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac03.mgf', mode='w')
idf04 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac04.mgf', mode='w')
idf05 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac05.mgf', mode='w')
idf06 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac06.mgf', mode='w')
idf07 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac07.mgf', mode='w')
idf08 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac08.mgf', mode='w')
idf09 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac09.mgf', mode='w')
idf10 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac10.mgf', mode='w')
idf11 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac11.mgf', mode='w')
idf12 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac12.mgf', mode='w')
idf13 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac13.mgf', mode='w')
idf14 = open('/data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac14.mgf', mode='w')
idf2frac = dict()
idf2frac[1]=idf01
idf2frac[2]=idf02
idf2frac[3]=idf03
idf2frac[4]=idf04
idf2frac[5]=idf05
idf2frac[6]=idf06
idf2frac[7]=idf07
idf2frac[8]=idf08
idf2frac[9]=idf09
idf2frac[10]=idf10
idf2frac[11]=idf11
idf2frac[12]=idf12
idf2frac[13]=idf13
idf2frac[14]=idf14
bins = [int(len(pep2spec)/14 * i) for i in range(14)]
idx2fracidx = np.digitize(range(len(pep2spec)), bins=bins)

for idx, ((p, c), mz2int) in enumerate(pep2spec.items()):
    if idx % 100000 == 0:
        print(idx, round(time.time() - start, 2), end='\r')
    idf = idf2frac[idx2fracidx[idx]]
    idf.write('BEGIN IONS\n')
    idf.write('TITLE='+str(idx+1)+' File:\"massIVE-KB_frac{}.mgf", NativeID:\"controllerType=0 controllerNumber=1 scan='.format(str(idx+1).zfill(2))+str(idx+1)+'\"\n')
    idf.write('PEPMASS='+str(cal_mz(p, c))+'\n')  # ' precursor_intensity'
    idf.write('CHARGE='+str(c)+'+\n')
    for mz, i in sorted(mz2int.items(), key=lambda x: x[0]):
        idf.write(str(mz)+' '+str(i)+'\n')
    idf.write('END IONS\n')

idf01.close()
idf02.close()
idf03.close()
idf04.close()
idf05.close()
idf06.close()
idf07.close()
idf08.close()
idf09.close()
idf10.close()
idf11.close()
idf12.close()
idf13.close()
idf14.close()

In [ ]:
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac01.mgf isa@166.104.110.50:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac02.mgf isa@166.104.110.51:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac03.mgf isa@166.104.110.52:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac04.mgf isa@166.104.110.53:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac05.mgf isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac06.mgf isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac07.mgf isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac08.mgf isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac09.mgf isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac10.mgf isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac11.mgf isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/

    
    # sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/uniprot_predMS2_frac12.mgf isa@166.104.112.62:/home/isa/Search/210831_SJH_prosit/210915/
    

In [ ]:
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac01.mgf isa@166.104.110.45:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac02.mgf isa@166.104.110.46:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac03.mgf isa@166.104.110.39:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac04.mgf isa@166.104.110.48:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac05.mgf isa@166.104.110.49:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac06.mgf isa@166.104.110.50:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac07.mgf isa@166.104.110.51:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac08.mgf isa@166.104.110.52:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac09.mgf isa@166.104.110.53:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac10.mgf isa@166.104.110.55:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac11.mgf isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac12.mgf isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac13.mgf isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/massIVE-KB_frac14.mgf isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/


In [ ]:
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/MSGFPlus_Params.txt isa@166.104.110.45:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/MSGFPlus_Params.txt isa@166.104.110.46:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/MSGFPlus_Params.txt isa@166.104.110.39:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/MSGFPlus_Params.txt isa@166.104.110.48:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/MSGFPlus_Params.txt isa@166.104.110.49:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/MSGFPlus_Params.txt isa@166.104.110.50:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/MSGFPlus_Params.txt isa@166.104.110.51:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/MSGFPlus_Params.txt isa@166.104.110.52:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/MSGFPlus_Params.txt isa@166.104.110.53:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/MSGFPlus_Params.txt isa@166.104.110.55:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/MSGFPlus_Params.txt isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/MSGFPlus_Params.txt isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/MSGFPlus_Params.txt isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/
sshpass -p "isa4986" scp /data/210827_SJH_prosit/211025STRIP_PEP/MSGFPlus_Params.txt isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/


In [ ]:
java -Xmx30000M -jar ../MSGFPlus.jar -s uniprot_frac01.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o uniprot_frac01.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i uniprot_frac01.mzid -o uniprot_frac01.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s uniprot_frac02.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o uniprot_frac02.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i uniprot_frac02.mzid -o uniprot_frac02.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s uniprot_frac03.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o uniprot_frac03.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i uniprot_frac03.mzid -o uniprot_frac03.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s uniprot_frac04.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o uniprot_frac04.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i uniprot_frac04.mzid -o uniprot_frac04.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s uniprot_frac05.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o uniprot_frac05.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i uniprot_frac05.mzid -o uniprot_frac05.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s uniprot_frac06.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o uniprot_frac06.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i uniprot_frac06.mzid -o uniprot_frac06.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s uniprot_frac07.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o uniprot_frac07.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i uniprot_frac07.mzid -o uniprot_frac07.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s uniprot_frac08.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o uniprot_frac08.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i uniprot_frac08.mzid -o uniprot_frac08.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s uniprot_frac09.mgf -d uniprot-proteome_UP000005640.fasta -o uniprot_frac09.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i uniprot_frac09.mzid -o uniprot_frac09.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s uniprot_frac10.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o uniprot_frac10.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i uniprot_frac10.mzid -o uniprot_frac10.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s uniprot_frac11.mgf -d uniprot-proteome_UP000005640.fasta -o uniprot_frac11.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i uniprot_frac11.mzid -o uniprot_frac11.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s uniprot_frac12.mgf -d uniprot-proteome_UP000005640.fasta -o uniprot_frac12.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i uniprot_frac12.mzid -o uniprot_frac12.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s uniprot_frac13.mgf -d uniprot-proteome_UP000005640.fasta -o uniprot_frac12.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i uniprot_frac12.mzid -o uniprot_frac12.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s uniprot_frac14.mgf -d uniprot-proteome_UP000005640.fasta -o uniprot_frac12.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i uniprot_frac12.mzid -o uniprot_frac12.tsv -showQValue 1 -showDecoy 1

In [ ]:
java -Xmx30000M -jar ../MSGFPlus.jar -s massIVE-KB_frac01.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o massIVE-KB_frac01.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i massIVE-KB_frac01.mzid -o massIVE-KB_frac01.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s massIVE-KB_frac02.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o massIVE-KB_frac02.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i massIVE-KB_frac02.mzid -o massIVE-KB_frac02.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s massIVE-KB_frac03.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o massIVE-KB_frac03.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i massIVE-KB_frac03.mzid -o massIVE-KB_frac03.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s massIVE-KB_frac04.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o massIVE-KB_frac04.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i massIVE-KB_frac04.mzid -o massIVE-KB_frac04.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s massIVE-KB_frac05.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o massIVE-KB_frac05.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i massIVE-KB_frac05.mzid -o massIVE-KB_frac05.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s massIVE-KB_frac06.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o massIVE-KB_frac06.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i massIVE-KB_frac06.mzid -o massIVE-KB_frac06.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s massIVE-KB_frac07.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o massIVE-KB_frac07.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i massIVE-KB_frac07.mzid -o massIVE-KB_frac07.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s massIVE-KB_frac08.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o massIVE-KB_frac08.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i massIVE-KB_frac08.mzid -o massIVE-KB_frac08.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s massIVE-KB_frac09.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o massIVE-KB_frac09.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i massIVE-KB_frac09.mzid -o massIVE-KB_frac09.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s massIVE-KB_frac10.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o massIVE-KB_frac10.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i massIVE-KB_frac10.mzid -o massIVE-KB_frac10.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s massIVE-KB_frac11.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o massIVE-KB_frac11.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i massIVE-KB_frac11.mzid -o massIVE-KB_frac11.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s massIVE-KB_frac12.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o massIVE-KB_frac12.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i massIVE-KB_frac12.mzid -o massIVE-KB_frac12.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s massIVE-KB_frac13.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o massIVE-KB_frac13.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i massIVE-KB_frac13.mzid -o massIVE-KB_frac13.tsv -showQValue 1 -showDecoy 1

java -Xmx30000M -jar ../MSGFPlus.jar -s massIVE-KB_frac14.mgf -d ../210831/uniprot-proteome_UP000005640.fasta -o massIVE-KB_frac14.mzid -protocol 5 -addFeatures 1 -conf MSGFPlus_Params.txt
java -Xmx30000M -cp ../MSGFPlus.jar edu.ucsd.msjava.ui.MzIDToTsv -i massIVE-KB_frac14.mzid -o massIVE-KB_frac14.tsv -showQValue 1 -showDecoy 1

In [ ]:
sshpass -p "isa4986" scp isa@166.104.110.45:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_frac01.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.46:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_frac02.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.39:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_frac03.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.48:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_frac04.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.49:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_frac05.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.50:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_frac06.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.51:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_frac07.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.52:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_frac08.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.53:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_frac09.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.55:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_frac10.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_frac11.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_frac12.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_frac13.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_frac14.tsv /data/210827_SJH_prosit/211025STRIP_PEP/


In [ ]:
head -n 1 massIVE-KB_frac01.tsv > real_massIVE-KB_result.tsv
for F in massIVE-KB_frac*.tsv ; do tail -n +2 ${F} >> real_massIVE-KB_result.tsv ; done


In [ ]:
sshpass -p "isa4986" scp isa@166.104.110.50:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_predMS2_frac01.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.51:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_predMS2_frac02.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.52:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_predMS2_frac03.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.53:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_predMS2_frac04.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_predMS2_frac05.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_predMS2_frac06.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_predMS2_frac07.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_predMS2_frac08.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
sshpass -p "isa4986" scp isa@166.104.110.56:/home/isa/Search/210831_SJH_prosit/210915/massIVE-KB_predMS2_frac10.tsv /data/210827_SJH_prosit/211025STRIP_PEP/
